In [1]:
import pandas as pd
import mlflow
import itertools

In [2]:
import sys
sys.path.append('../../src')

from search import search

/home/felix/tmp/lexum-test/lab/.venv/lib/python3.12/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/home/felix/tmp/lexum-test/lab/.venv/lib/python3.12/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

basque.txt:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

azerbaijani.txt:   0%|          | 0.00/967 [00:00<?, ?B/s]

arabic.txt:   0%|          | 0.00/6.35k [00:00<?, ?B/s]

...

spanish.txt:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

slovene.txt:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

hinglish.txt:   0%|          | 0.00/5.96k [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

hungarian.txt:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

In [3]:
mlflow.set_tracking_uri("../../mlruns")
mlflow.set_experiment("information_retrieval")

2025/05/15 23:08:59 INFO mlflow.tracking.fluent: Experiment with name 'information_retrieval' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/felix/tmp/lexum-test/lab/notebooks/03_experiments/../../mlruns/630467622743382322', creation_time=1747364939641, experiment_id='630467622743382322', last_update_time=1747364939641, lifecycle_stage='active', name='information_retrieval', tags={}>

In [4]:
def run_search(
    query,
    method,
    k=5,
    filter_by_entity=False,
    do_rerank=False,
    do_graph_expansion=False
):
    docs = search(
        query,
        method=method,
        k=k,
        filter_by_entity=filter_by_entity,
        do_rerank=do_rerank,
        do_graph_expansion=do_graph_expansion
    )

    docs = [doc.id for doc in docs]
    return docs

In [5]:
def run_experiment(df, method, dataset_name, k=5, filter_by_entity=False, do_rerank=False, do_graph_expansion=False):
    df = df.copy()

    with mlflow.start_run() as run:
        df['retrieved'] = df.apply(
            run_search,
            method=method,
            k=k,
            filter_by_entity=filter_by_entity,
            do_rerank=do_rerank,
            do_graph_expansion=do_graph_expansion,
            axis=1
        )

        mlflow.log_param("k", k)
        mlflow.log_param("dataset_name", dataset_name)
        mlflow.log_param("method", method)
        mlflow.log_param("filter_by_entity", filter_by_entity)
        mlflow.log_param("do_rerank", do_rerank)
        mlflow.log_param("do_graph_expansion", do_graph_expansion)

        mlflow.evaluate(
            data=df,
            model_type="retriever",
            targets="target",
            predictions="retrieved",
            evaluators="default",
            extra_metrics=[
                mlflow.metrics.recall_at_k(1),
                mlflow.metrics.recall_at_k(5),
                mlflow.metrics.ndcg_at_k(1),
                mlflow.metrics.ndcg_at_k(5),
                mlflow.metrics.precision_at_k(1),
                mlflow.metrics.precision_at_k(5),
            ],
        )

In [6]:
def run_all_experiments(df, dataset_name):
    params = itertools.product(
        [1, 5, 10],
        ["bm25", "dense", "hybrid"],
        [True, False],
        [True, False],
    )

    for k, m, filter_by_entity, do_rerank in params:
        run_experiment(df, m, dataset_name, k, filter_by_entity, do_rerank)


In [9]:
df = pd.read_parquet('../../data/retrieval_questions.parquet')
df['target'] = df.uuid.apply(lambda x: [x])
df['entities'] = df.entities.apply(
    lambda x: {
        k: [v_ for v_ in v if v_ is not None]
        for k, v in x.items()
        if v is not None and len(v) > 0
    }
)

run_all_experiments(df, "simple")

2025/05/15 23:11:55 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:11:56 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:11:57 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:11:57 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:11:59 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:12:00 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:12:02 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:12:02 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:12:05 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:12:06 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on fir

In [10]:
df = pd.read_parquet('../../data/retrieval_questions_detail.parquet')

df['target'] = df.uuid.apply(lambda x: [x])
df['entities'] = df.entities.apply(
    lambda x: {
        k: [v_ for v_ in v if v_ is not None]
        for k, v in x.items()
        if v is not None and len(v) > 0
    }
)

run_all_experiments(df, "detail")

2025/05/15 23:14:08 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:09 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:10 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:11 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:13 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:14 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:15 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:16 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:19 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:14:20 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on fir

In [11]:
df = pd.read_parquet('../../data/retrieval_questions_multichunks.parquet')

df['target'] = df.target.apply(lambda x: x.tolist())
df['entities'] = df.entities.apply(
    lambda x: {
        k: [v_ for v_ in v if v_ is not None]
        for k, v in x.items()
        if v is not None and len(v) > 0
    }
)

run_all_experiments(df, "multichunks")

2025/05/15 23:16:25 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:25 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:27 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:27 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:29 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:30 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:32 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:32 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:34 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/05/15 23:16:36 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on fir